# 03. Despliegue del Modelo e Integración RAG (Fase de Validación)

## Objetivos de este Notebook
1.  **Carga del SLM:** Desplegar el modelo **Microsoft Phi-3-mini-4k-instruct** en la GPU (RTX 4090).
2.  **Conexión Distribuida:** Establecer comunicación con Elasticsearch (alojado en CPU/Valencia) mediante túnel SSH.
3.  **Pipeline RAG:** Implementar la función de búsqueda y generación con configuración estricta (**Top-k=1**, **Match Perfecto**) para mitigar alucinaciones.
4.  **Validación Técnica:** Realizar pruebas manuales de cordura (Sanity Checks) para asegurar que el sistema recupera noticias reales y admite ignorancia cuando no hay datos.

## Requisitos Previos
* Túnel SSH activo en terminal: `ssh -L 9250:localhost:9250 javierruiz@valencia...`
* Elasticsearch corriendo en el servidor remoto.

In [1]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# 1. Configuración de Hardware
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Hardware detectado: {torch.cuda.get_device_name(0) if device == 'cuda' else 'CPU'}")

# 2. Selección de Modelo (Empezamos con un SLM para la prueba rápida)
MODEL_ID = "microsoft/Phi-3-mini-4k-instruct"

print(f"Cargando modelo: {MODEL_ID}...")

# 3. Carga del Tokenizer y el Modelo 
# Importante: Ponemos trust_remote_code=False para usar la versión estable instalada
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID, trust_remote_code=False)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    device_map="auto",             # Distribuye automáticamente en la GPU
    torch_dtype=torch.float16    # Usamos precisión media para ahorrar VRAM y ganar velocidad
)

# 4. Crear Pipeline de Generación
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
)

print("\n ¡Modelo SLM cargado y listo para inferencia!")

/home/javierruiz/miniconda3/envs/environment/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Hardware detectado: NVIDIA GeForce RTX 4090
Cargando modelo: microsoft/Phi-3-mini-4k-instruct...


`torch_dtype` is deprecated! Use `dtype` instead!
Loading weights: 100%|██████████| 195/195 [00:01<00:00, 125.49it/s, Materializing param=model.norm.weight]                              



 ¡Modelo SLM cargado y listo para inferencia!


In [2]:
# 5. Prueba de Inferencia (Sin RAG todavía)
# Vamos a ver si el "cerebro" funciona por sí solo.

messages = [
    {"role": "user", "content": "¿Podrías explicarme brevemente qué es la inflación económica?"},
]

# Configuración de generación
generation_args = {
    "max_new_tokens": 150,     # Que no escriba una novela, solo un párrafo
    "return_full_text": False, # Que no repita la pregunta
    "temperature": 0.1,        # Creatividad baja (queremos datos, no poesía)
    "do_sample": True,
}

print(" Generando respuesta...")
output = pipe(messages, **generation_args)
print("\n" + "="*50)
print(output[0]['generated_text'])
print("="*50)

Passing `generation_config` together with generation-related arguments=({'temperature', 'do_sample', 'max_new_tokens'}) is deprecated and will be removed in future versions. Please pass either a `generation_config` object OR all generation parameters explicitly, but not both.
Both `max_new_tokens` (=150) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 Generando respuesta...

 La inflación económica es un fenómeno que se refiere a la tasa a la que el nivel general de precios de bienes y servicios está aumentando, y, por ende, el poder adquisitivo está disminuyendo. En otras palabras, a medida que la inflación aumenta, cada unidad de moneda compra menos bienes y servicios. La inflación puede ser causada por diversos factores, incluyendo un aumento en la oferta de dinero, un aumento en los costos de producción, o una disminución en la demanda de bienes y servicios. Las autoridades monetarias, como el Banco


In [3]:
from elasticsearch import Elasticsearch

# Ahora que las versiones coinciden, la configuración es estándar y sencilla
es = Elasticsearch("http://127.0.0.1:9250")

try:
    if es.ping():
        print("PING EXITOSO")
        print(f" Conectado a: {es.info()['name']}")
    else:
        print("❌ El servidor no responde. ¿Está el túnel abierto?")
except Exception as e:
    print(f"❌ Error inesperado: {e}")

PING EXITOSO
 Conectado a: valencia


In [4]:
def ask_rag(query, top_k=1): 
    """
    Función RAG para TFG: Recupera 1 noticia y genera respuesta basada estrictamente en ella.
    Retorna un diccionario con los datos separados.
    """
    # --- A. BÚSQUEDA ---
    search_payload = {
        "size": top_k,
        "query": {
            "multi_match": {
                "query": query,
                "fields": ["title^3", "body"],
                "fuzziness": 0
            }
        },
        "_source": ["title", "body", "date", "source"]
    }
    
    try:
        response = es.search(index="noticias_tfg", body=search_payload)
        hits = response['hits']['hits']
    except Exception as e:
        return {"error": f"Error en Elasticsearch: {e}"}
    
    if not hits:
        return {"error": " NO ENCONTRADO: No hay ninguna noticia que coincida exactamente con la búsqueda."}

    # --- B. EXTRACCIÓN ---
    doc = hits[0]['_source']
    contexto_unico = f"""
    TITULO: {doc.get('title')}
    FECHA: {doc.get('date', 'Desconocida')}
    FUENTE: {doc.get('source', 'Desconocida')}
    CONTENIDO: {doc.get('body')}
    """

    # --- C. PROMPT ---
    messages = [
        {"role": "user", "content": f"""
        Eres un sistema de verificación de datos (Fact-Checking). 
        Tu objetivo es responder a la pregunta usando ÚNICAMENTE el texto que te proporciono abajo.

        REGLAS CRÍTICAS:
        1. Si la respuesta no aparece en el texto, responde exactamente: "No tengo información suficiente en mis archivos".
        2. No utilices conocimiento externo.
        3. No menciones otras noticias que no sean la proporcionada.

        ### TEXTO DE REFERENCIA:
        {contexto_unico}
        
        ### PREGUNTA:
        {query}
        """}
    ]

    # --- D. GENERACIÓN ---
    outputs = pipe(
        messages,
        max_new_tokens=256,
        do_sample=False,
        temperature=0.0, 
    )
    
    # --- E. RETORNO ESTRUCTURADO  ---
    # Extraemos solo el texto del último mensaje 
    respuesta_limpia = outputs[0]['generated_text'][-1]['content']
    
    return {
        "titulo": doc.get('title'),
        "contenido": doc.get('body'),
        "fecha": doc.get('date'),
        "fuente": doc.get('source'),
        "respuesta_rag": respuesta_limpia # Aquí va solo el texto que querías
    }

print("✅ Sistema RAG (k=1) reconfigurado para devolver datos separados.")

✅ Sistema RAG (k=1) reconfigurado para devolver datos separados.


In [ ]:
pregunta_tfg = "¿Qué aranceles puso Trump a China?"

print(f" PREGUNTA: {pregunta_tfg}\n")

# 1. Prueba SIN RAG 
print("--- RESPUESTA SLM (SIN RAG) ---")
res_base = pipe([{"role": "user", "content": pregunta_tfg}], max_new_tokens=150)
print(res_base[0]['generated_text'][-1]['content']) # Solo el texto generado, sin repetir la pregunta

print("\n")

# Ejecutamos el RAG
datos = ask_rag(pregunta_tfg)
print("--- RESPUESTA SISTEMA RAG ---")
# 1. Imprimimos Título
print(f"NOTICIA: {datos['titulo']}")

# 2. Imprimimos Contenido (Recortado un poco para no llenar pantalla)
print(f"TEXTO: {datos['contenido'][:300]} (...)")
print("FUENTE: " + datos['fuente'])
# 3. Imprimimos SOLO la respuesta limpia del RAG
print(f"RESPUESTA: {datos['respuesta_rag']}")

Passing `generation_config` together with generation-related arguments=({'max_new_tokens'}) is deprecated and will be removed in future versions. Please pass either a `generation_config` object OR all generation parameters explicitly, but not both.
Both `max_new_tokens` (=150) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 PREGUNTA: ¿Qué aranceles puso Trump a China?

--- RESPUESTA SLM (SIN RAG) ---


The following generation flags are not valid and may be ignored: ['temperature']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Both `max_new_tokens` (=256) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 El presidente Donald Trump impuso aranceles a China en 2018 como parte de una política comercial destinada a reducir el déficit comercial estadounidense y proteger las industrias estadounidenses de la competencia china. Los aranceles, conocidos como "tasa de penalización", aumentaron progresivamente, afectando una amplia gama de productos. Entre ellos figuraban equipos de computadora, teléfonos móviles, automóviles, paneles solares, y bienes electrónicos, entre otros. Este enfoque comercial empezó a tensar las relaciones económicas entre Estados Unidos y China, lo


--- RESPUESTA SISTEMA RAG ---
NOTICIA: Aranceles de Trump a China alcanzan el 145%
TEXTO: La Casa Blanca precisó que el aumento del 125% de aranceles a China se suma al 20% vigente desde principios de marzo. (...)
FUENTE: Excélsior
RESPUESTA:  El presidente Trump puso aranceles a China que suman un 125% al 20% vigente desde principios de marzo.


In [7]:
pregunta_tfg = "¿Cuál dijo Pedro Sánchez que era su principal prioridad en la clausura del 41 Congreso Federal del PSOE?"

print(f" PREGUNTA: {pregunta_tfg}\n")

# 1. Prueba SIN RAG 
print("--- RESPUESTA SLM (SIN RAG) ---")
res_base = pipe([{"role": "user", "content": pregunta_tfg}], max_new_tokens=150)
print(res_base[0]['generated_text'][-1]['content']) # Solo el texto generado, sin repetir la pregunta

print("\n")

# Ejecutamos el RAG
datos = ask_rag(pregunta_tfg)
print("--- RESPUESTA SISTEMA RAG ---")
# 1. Imprimimos Título
print(f"NOTICIA: {datos['titulo']}")

# 2. Imprimimos Contenido (Recortado un poco para no llenar pantalla)
print(f"TEXTO: {datos['contenido']}")
print("FUENTE: " + datos['fuente'])
# 3. Imprimimos SOLO la respuesta limpia del RAG
print(f"RESPUESTA: {datos['respuesta_rag']}")

Both `max_new_tokens` (=150) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 PREGUNTA: ¿Cuál dijo Pedro Sánchez que era su principal prioridad en la clausura del 41 Congreso Federal del PSOE?

--- RESPUESTA SLM (SIN RAG) ---


Both `max_new_tokens` (=256) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


 Durante la clausura del 41 Congreso Federal del PSOE, el entonces secretario general Pedro Sánchez, en un discurso público, enfatizó la necesidad de trabajar por la transición exitosa hacia la convocatoria de las elecciones generales de 2023, que eran una de sus principales prioridades. Dijo que era fundamental para el partido y para la nación avanzar hacia un proceso democrático que permitiera al pueblo español elegir a sus representantes.


--- RESPUESTA SISTEMA RAG ---
NOTICIA: Todos los señalados por Pedro Sánchez en el Congreso Federal del PSOE
TEXTO: Pedro Sánchez aseguró en la clausura el domingo en Sevilla del 41 Congreso Federal del PSOE que su «principal prioridad» sería revalidar su propio mandato en 2027 y, al mismo tiempo, recuperar el poder territorial que perdieron los socialistas frente al PP ... en 2023.Andalucía Juan Espadas De candidato contra Díaz a discutidoEl líder del PSOE andaluz ha pasado en dos años de ser el elegido por Ferraz para derrocar a Susana Díaz en 